<a href="https://colab.research.google.com/github/kelly1325/Python/blob/main/KellyG_module_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Module 4

The following Python script scrapes payroll data from the following website: http://kanview.ks.gov/

"KanView is an online solution that brings better visibility, openness, and accountability to Kansas State Government. KanView contains data related to Government spending and income, allowing Kansas taxpayers an inside look at Government financial activity."

The script scrapes the payroll data for the Fort Hays State University (https://www.fhsu.edu/) using Selenium drive along with Beautiful soup parsing library. It stores the scraped results in a Pandas DataFrame. It converts the DataFrame into a corresponding JSON object. It finally writes the JSON object into a json file in your local file system.

In [ ]:
%%bash
chmod 777 /tmp
mkdir data
apt-get update --allow-unauthenticated 
apt-get update -y --fix-missing 
pip install selenium
apt-get install chromium-chromedriver -y --fix-missing
pip install joblib
apt-get update --fix-missing

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,396 kB]
Hit:12 http://ppa.launchpad.net

In [ ]:
## Headless Browser with BeautifulSoup
import selenium
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

import re                      # for handling regular expressions
import pandas as pd            # using pandas library
from tabulate import tabulate  # tabulate, which takes a list of lists or another tabular data type as the first argument, and outputs a nicely formatted plain-text table


options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.binary_location = '/usr/bin/chromium-browser'

driver = webdriver.Chrome(executable_path='/usr/bin/chromedriver', options = options)
driver.get("http://kanview.ks.gov/PayRates/PayRates_Agency.aspx")


#After opening the url above, Selenium clicks the specific agency link, in this case FHSU (Fort Hays State University)
python_button = driver.find_element_by_id('MainContent_uxLevel1_Agencies_uxAgencyBtn_87') #FHSU
python_button.click() #click fhsu link


#Selenium hands the page source to Beautiful Soup
soup_level1=BeautifulSoup(driver.page_source, 'lxml')


datalist = [] #empty list
x = 0 #counter

#Beautiful Soup finds all Job Title links on the agency page and the loop begins
for link in soup_level1.find_all('a', id=re.compile("^MainContent_uxLevel2_JobTitles_uxJobTitleBtn_")):
    
    #Selenium visits each Job Title page
    python_button = driver.find_element_by_id('MainContent_uxLevel2_JobTitles_uxJobTitleBtn_' + str(x))
    python_button.click() #click link
    
    #Selenium hands of the source of the specific job page to Beautiful Soup
    soup_level2=BeautifulSoup(driver.page_source, 'lxml')

    #Beautiful Soup grabs the HTML table on the page
    table = soup_level2.find_all('table')[0]
    
    #Giving the HTML table to pandas to put in a dataframe object
    df = pd.read_html(str(table),header=0)
    
    #Store the dataframe in a list
    datalist.append(df[0])
    
    #Ask Selenium to click the back button
    driver.execute_script("window.history.go(-1)") 
    
    #increment the counter variable before starting the loop over
    x += 1
    
    #end loop block
    
#loop has completed

#end the Selenium browser session
driver.quit()


In [ ]:
#combine all pandas dataframes in the list into one big dataframe
result = pd.concat([pd.DataFrame(datalist[i]) for i in range(len(datalist))],ignore_index=True)

#convert the pandas dataframe to JSON
json_records = result.to_json(orient='records')

#pretty print to CLI with tabulate
#converts to an ascii table
print(tabulate(result, headers=["Employee Name","Job Title","Overtime Pay","Total Gross Pay"],tablefmt='psql'))


+------+-----------------------------------------------+--------------------------------+----------------+-------------------+
|      | Employee Name                                 | Job Title                      | Overtime Pay   | Total Gross Pay   |
|------+-----------------------------------------------+--------------------------------+----------------+-------------------|
|    0 | Lichtenauer,Brianna Leigh                     | Academic Accommodations Advs   | $0.00          | $39,803.02        |
|    1 | Washington,Dakei                              | Academic Accommodations Advs   | $0.00          | $14,057.64        |
|    2 | Wilson Merriman,Sarah June                    | Academic Accommodations Advs   | $0.00          | $36,419.40        |
|    3 | Adams,Madelyn Jeanne                          | Academic Advisor               | $0.00          | $39,999.96        |
|    4 | Adams,Tanay Maurisha                          | Academic Advisor               | $0.00          | $37,

In [ ]:
#open, write and download the file to local file system
from google.colab import files

with open('payroll_data.json', 'w') as f:
  f.write(json_records)

files.download('payroll_data.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
result

,Employee Name,Job Title,Overtime Pay,Total Gross Pay
0,"Lichtenauer,Brianna Leigh",Academic Accommodations Advs,$0.00,"$39,803.02"
1,"Washington,Dakei",Academic Accommodations Advs,$0.00,"$14,057.64"
2,"Wilson Merriman,Sarah June",Academic Accommodations Advs,$0.00,"$36,419.40"
3,"Adams,Madelyn Jeanne",Academic Advisor,$0.00,"$39,999.96"
4,"Adams,Tanay Maurisha",Academic Advisor,$0.00,"$37,797.25"
...,...,...,...,...
4601,"Touyz,Paul Mark",Visiting Assistant Professor,$0.00,"$56,960.52"
4602,"Zheng,Guangqu",Visiting Assistant Professor,$0.00,"$55,421.17"
4603,"McKinney,Kristy Louise",Visitor/Guest Services Coord,$0.00,"$17,531.04"
4604,"Bryant,Loyd W.",Welder Senior,$0.00,"$49,344.30"


In [ ]:
result['Total Gross Pay']

Job Title
Academic Accommodations Advs   3980302.00
Academic Accommodations Advs   1405764.00
Academic Accommodations Advs   3641940.00
Academic Advisor               3999996.00
Academic Advisor               3779725.00
                                  ...    
Visiting Assistant Professor   5696052.00
Visiting Assistant Professor   5542117.00
Visitor/Guest Services Coord   1753104.00
Welder Senior                  4934430.00
Welder Senior                  4748577.00
Name: Total Gross Pay, Length: 4606, dtype: float64

### Question 2.

Using .decribe() method in Pandas (https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.describe.html), generate descriptive statistics for Total Gross Pay and Overtime Pay.



In [ ]:
# Answer:

import pandas as pd
import numpy as np

#pd.options.display.float_format = "{:.2f}".format
result["Total Gross Pay"] = result["Total Gross Pay"].replace('[\$\,\0]',"",regex=True).astype(float)
result[["Total Gross Pay"]].describe()


,Total Gross Pay
count,4606.00
mean,6888660.55
std,5160229.92
min,45132.00
25%,4067585.25
50%,5550139.00
75%,8263136.75
max,147692314.00


In [ ]:
result["Overtime Pay"] = result["Overtime Pay"].replace('[\$\,\0]',"",regex=True).astype(float)
result[["Overtime Pay"]].describe()

,Overtime Pay
count,4606.00
mean,97.83
std,3160.76
min,0.00
25%,0.00
50%,0.00
75%,0.00
max,144554.00


### Question 3.
Using Pandas, calculate the average Total Gross Pay for Education Program Coordinator.

In [ ]:
# Answer: 


In [ ]:
import pandas as pd
import numpy as np

result

,Employee Name,Overtime Pay,Total Gross Pay
Job Title,,,
Academic Accommodations Advs,"Lichtenauer,Brianna Leigh",$0.00,"$39,803.02"
Academic Accommodations Advs,"Washington,Dakei",$0.00,"$14,057.64"
Academic Accommodations Advs,"Wilson Merriman,Sarah June",$0.00,"$36,419.40"
Academic Advisor,"Adams,Madelyn Jeanne",$0.00,"$39,999.96"
Academic Advisor,"Adams,Tanay Maurisha",$0.00,"$37,797.25"
...,...,...,...
Visiting Assistant Professor,"Touyz,Paul Mark",$0.00,"$56,960.52"
Visiting Assistant Professor,"Zheng,Guangqu",$0.00,"$55,421.17"
Visitor/Guest Services Coord,"McKinney,Kristy Louise",$0.00,"$17,531.04"


In [ ]:
EDU=result.loc[["Education Program Coordinator"], ["Total Gross Pay"]]
EDU

,Total Gross Pay
Job Title,
Education Program Coordinator,"$26,823.72"
Education Program Coordinator,"$50,817.27"
Education Program Coordinator,"$44,068.90"
Education Program Coordinator,"$10,455.00"
Education Program Coordinator,"$49,961.13"
...,...
Education Program Coordinator,"$57,363.47"
Education Program Coordinator,"$19,262.25"
Education Program Coordinator,"$36,923.00"


In [ ]:
EDU["Total Gross Pay"] = EDU["Total Gross Pay"].replace('[\$\,\0]',"",regex=True).astype(float)
EDU[["Total Gross Pay"]].mean()

Total Gross Pay   47259.57
dtype: float64